# 제2 고지 : 자연스러운  코드로 
## STEP 20 : 연산자 오버로드(1)

이전 단계에서는 변수 사용성 개선을 통해 '투명한 상자'로 만드는 작업을 진행했다.  
하지만, 아직 `+`,`*`,`/`,`-`,`**` 와 같은 연산자를 지원하지 않아, `y=a*b` 와 같이 손쉬운 함수 정의가 어렵다.  
이번 단계에서는 `+`,`*` 연산자를 지원하도록 구현해본다.

### 20.1 Mul 클래스 구현 

![image](../assets/%EA%B7%B8%EB%A6%BC%2020-1.png)

곱셈의 미분은 $y=x_0x_1$ 일때,  $\begin{bmatrix}\frac{\partial y}{\partial x_0} & \frac{\partial y}{\partial x_1} \end{bmatrix} = \begin{bmatrix} x_1 & x_0 \end{bmatrix}$  이다.

위의 그림에서 보듯, 우리가 관심을 가지는 것은 **스칼라를 출력하는 합성함수** $\mathcal{L}$  이므로 최종출력 $\mathcal{L}$ 의 각 변수에 대한 미분인 $\begin{bmatrix}\frac{\partial \mathcal{L}}{\partial x_0} & \frac{\partial \mathcal{L}}{\partial x_1} \end{bmatrix} = \begin{bmatrix}x_1\frac{\partial \mathcal{L}}{\partial y} & x_0\frac{\partial \mathcal{L}}{\partial y} \end{bmatrix} $ 이다.  
이를 바탕으로 `Mul` 클래스를 구현한다. 


In [8]:
import weakref
import numpy as np 


class Variable:
    def __init__(self, data,name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name  # 변수 구분을 위한 `이름` 설정
        self.grad = None
        self.creator = None
        self.generation = 0
        
    @property
    def ndim(self):
        return self.data.ndim
    @property
    def shape(self):
        return self.data.shape
    @property
    def size(self):
        return self.data.size
    @property
    def dtype(self):
        return self.data.dtype

    #############################
    def __len__(self):
        return len(self.data)
    
    def __repr__(self):
        if self.data is None:
            return "variable(None)"
        p = str(self.data).replace("\n","\n"+" "*9)
        return 'variable('+p+')'
        
    #############################
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None
        
    def backward(self,retain_grad=False):  # `retain_grad` 추가 
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref이기 때문에 y()로 호출

def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = [weakref.ref(output) for output in outputs]
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()

class Add(Function):
    def forward(self, x0,x1):
        y = x0 + x1
        return y
    def backward(self, gy):
        # 역전파시 , 입력이 1개 , 출력이 2개 
        return gy,gy 
    
def add(x0,x1):
    return Add()(x0,x1)


In [9]:
class Mul(Function):
    def forward(self, x0, x1):
        y = x0 * x1
        return y

    def backward(self, gy):
        x0, x1 = self.inputs[0].data, self.inputs[1].data
        return gy * x1, gy * x0
    
def mul(x0,x1):
    return Mul()(x0,x1)

$$
y=a*b+c\\
\Rightarrow \begin{bmatrix} \frac{\partial y}{\partial a} & \frac{\partial y}{\partial b} & \frac{\partial y}{\partial c}\end{bmatrix}=\begin{bmatrix} b & a & 1\end{bmatrix}
$$


In [12]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

y = add(mul(a,b),c)
y.backward() 

print(y)
print(f"dy/da : {a.grad} , dy/db : {b.grad}, dy/dc : {c.grad}")


variable(7.0)
dy/da : 2.0 , dy/db : 3.0, dy/dc : 1.0


### 20.2 연산자 오버로드

앞서 살펴봤듯이, 아주 간단한 함수 $y=a*b+c$ 임에도 불구하고 코드상의 구현은 `y = add(mul(a,b),c)`으로 간단하지 않다. 만약 더 복잡한 함수가 주어진다면 코드 역시 복잡해 질것이다.  
이를 해결하기 위해 `__add__` 와 `__mul__` 특수 메서드를 정의하고  **연산자 오버로드** 를 이용해 `+`,`*` 를 사용할 수 있도록 확장한다.


![image](../assets/%EA%B7%B8%EB%A6%BC%2020-2.png)

우선 `*` 연산을 정의하면 위의 그림과 같이 `*` 왼쪽에 `a`가 인수 `self` 에 전달되고, 오른쪽의 `b`가 `other`에 전달된다.  
여기서 주목해야 할 것은 호출 과정인데, `y = a*b` 를 실행하면 다음과 같이 이뤄진다.

1. 인스턴스 `a` 의 `__mul__` 을 호출한다.

2. 만약 정의되어 있지 않다면, 인스턴스 `b`의 `__rmul__` 를 호출한다.

In [13]:
class Variable:
    def __init__(self, data,name=None):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.name = name  # 변수 구분을 위한 `이름` 설정
        self.grad = None
        self.creator = None
        self.generation = 0
        
    @property
    def ndim(self):
        return self.data.ndim
    @property
    def shape(self):
        return self.data.shape
    @property
    def size(self):
        return self.data.size
    @property
    def dtype(self):
        return self.data.dtype

    def __len__(self):
        return len(self.data)
    
    def __repr__(self):
        if self.data is None:
            return "variable(None)"
        p = str(self.data).replace("\n","\n"+" "*9)
        return 'variable('+p+')'
        
    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None
        
    def backward(self,retain_grad=False):  # `retain_grad` 추가 
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)

            if not retain_grad:
                for y in f.outputs:
                    y().grad = None  # y is weakref이기 때문에 y()로 호출
               
    ######################################
    def __mul__(self,other):
        return mul(self,other)
    def __add__(self,other):
        return add(self,other)
    ######################################
    

참고로, 다음과 같이 더 간단하게 구현할 수 있다.
```python
Variable.__mul__=mul
Variable.__add__=add
```

In [14]:
a = Variable(np.array(3.0))
b = Variable(np.array(2.0))
c = Variable(np.array(1.0))

# y = add(mul(a,b),c)
y = a*b + c
y.backward() 

print(y)
print(f"dy/da : {a.grad} , dy/db : {b.grad}, dy/dc : {c.grad}")


variable(7.0)
dy/da : 2.0 , dy/db : 3.0, dy/dc : 1.0
